### **Bibliotecas**

In [81]:
import pandas as pd
import numpy as np
import matplotlib
import seaborn as sns
import sklearn

bibliotecas = {"Pandas":pd, "Numpy":np, "Matplotlib":matplotlib, "Seaborn":sns, "Sklearn":sklearn}

for b, v in zip(bibliotecas.items(), bibliotecas.values()):
    print(f"A biblioteca {b[0]} está na versão {v.__version__}")

A biblioteca Pandas está na versão 1.5.3
A biblioteca Numpy está na versão 1.24.3
A biblioteca Matplotlib está na versão 3.7.1
A biblioteca Seaborn está na versão 0.12.2
A biblioteca Sklearn está na versão 1.4.0


### **Pré-Processamento das colunas categóricas:**

#### **Não utilizadas**:
- Street, Utilities, LandContour*, LandSlope, Condition1, Condition2, BldgType, RoofMatl, Heating, Electrical, Functional, GarageType, GarageCond, PavedDrive.

#### **OneHotEncoder**;
- MSZoning, LotConfig, LotShape, Neighborhood, HouseStyle (func_e_onehot), RoofStyle, Exterior1st(func_e_onehot), Exterior2nd(func_e_onehot), MasVnrType, Foundation, BsmtFinType1, BsmtFinType2,.

#### **OrdinalEncoder**:
- ExterQual, ExterCond, BsmtQual, BsmtCond, BsmtExposure, CentralAir, KitchenQual, FireplaceQu, GarageFinish.

### **Pré-Processamento das colunas numéricas**

#### **Não utilizadas** - Baixas correlações com o preço da casa < 0.3
- 'KitchenAbvGr', 'EnclosedPorch', 'MSSubClass', 'OverallCond', 'YrSold', 'LowQualFinSF', 
'MiscVal', 'BsmtHalfBath', 'BsmtFinSF2', '3SsnPorch', 'MoSold', 'ScreenPorch', 'BedroomAbvGr', 'BsmtUnfSF', 'GarageYrBlt', 'LotArea', 'HalfBath'
#### **MinMaxScaler (ou_onehot)**:
- Overalqual, OveralCond, Fireplaces, GarageCars, BsmtFullBath, FullBath, TotRmsAbvGrd
#### **RobustScaler**:
- LotFrontage, TotalBsmtSF, 1stFlrSF, GrLivArea, GarageArea
#### **PowerTransformer**:
- YearBuilt, YearRemodeAdd, MasVnrArea, BsmtFinSF1, BsmtFinSF2, WoodDeckSF, OpenPorchSF, 2ndFlrSF

**Observação**: O modelo que usarei é baseado em árvores de decisão, ou seja, essa mudança nas escalas dos dados não será influente no resultado do modelo. Porém, optei por definir essas colunas no notebook anterior e relembrar neste. Isso, com o intuito de demonstrar entendimento e também ter uma certa referência no caso da preferência por modelos que são influenciados por escalas.

### **Criando o pré-processamento e o modelo manualmente**

In [82]:
from sklearn.preprocessing import MinMaxScaler, RobustScaler, PowerTransformer
from sklearn.preprocessing import OneHotEncoder 
from sklearn.preprocessing import OrdinalEncoder 
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import KFold, cross_val_score
from sklearn.pipeline import Pipeline 

In [83]:
base = pd.read_csv("../dados/train_notnull.csv")
pd.set_option('display.max_columns',100)

# Colunas categóricas
base = base.drop(['Street', 'Utilities', 'LandContour', 'LandSlope', 'Condition1', 'Condition2', 
                'BldgType', 'RoofMatl', 'Heating', 'Electrical', 'Functional', 'GarageType', 
                'GarageCond', 'PavedDrive','CentralAir','SaleType','SaleCondition'],axis=1)
# Colunas numéricas
base = base.drop(['KitchenAbvGr', 'EnclosedPorch', 'MSSubClass', 'OverallCond', 'YrSold', 'LowQualFinSF', 
                  'MiscVal', 'BsmtHalfBath', 'BsmtFinSF2', '3SsnPorch', 'MoSold', 'ScreenPorch', 'BedroomAbvGr',
                  'BsmtUnfSF', 'GarageYrBlt', 'LotArea', 'HalfBath','BsmtFinType2'],axis=1)
base.head()

,MSZoning,LotFrontage,LotShape,LotConfig,Neighborhood,HouseStyle,OverallQual,YearBuilt,YearRemodAdd,RoofStyle,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,TotalBsmtSF,HeatingQC,1stFlrSF,2ndFlrSF,GrLivArea,BsmtFullBath,FullBath,KitchenQual,TotRmsAbvGrd,Fireplaces,FireplaceQu,GarageFinish,GarageCars,GarageArea,GarageQual,WoodDeckSF,OpenPorchSF,SalePrice
0,RL,65.0,Reg,Inside,CollgCr,2Story,7,2003,2003,Gable,VinylSd,VinylSd,BrkFace,196.0,Gd,TA,PConc,Gd,TA,No,GLQ,706,856,Ex,856,854,1710,1,2,Gd,8,0,No Fireplaces,RFn,2,548,TA,0,61,208500
1,RL,80.0,Reg,FR2,Veenker,1Story,6,1976,1976,Gable,MetalSd,MetalSd,None,0.0,TA,TA,CBlock,Gd,TA,Gd,ALQ,978,1262,Ex,1262,0,1262,0,2,TA,6,1,TA,RFn,2,460,TA,298,0,181500
2,RL,68.0,IR1,Inside,CollgCr,2Story,7,2001,2002,Gable,VinylSd,VinylSd,BrkFace,162.0,Gd,TA,PConc,Gd,TA,Mn,GLQ,486,920,Ex,920,866,1786,1,2,Gd,6,1,TA,RFn,2,608,TA,0,42,223500
3,RL,60.0,IR1,Corner,Crawfor,2Story,7,1915,1970,Gable,Wd Sdng,Wd Shng,None,0.0,TA,TA,BrkTil,TA,Gd,No,ALQ,216,756,Gd,961,756,1717,1,1,Gd,7,1,Gd,Unf,3,642,TA,0,35,140000
4,RL,84.0,IR1,FR2,NoRidge,2Story,8,2000,2000,Gable,VinylSd,VinylSd,BrkFace,350.0,Gd,TA,PConc,Gd,TA,Av,GLQ,655,1145,Ex,1145,1053,2198,1,2,Gd,9,1,TA,RFn,3,836,TA,192,84,250000


In [84]:
from sklearn.model_selection import train_test_split
X = base.drop('SalePrice',axis=1)
y = base.SalePrice
xtrain, xtest, ytrain, ytest = train_test_split(X,y, test_size=0.15, random_state=0)

#### **Criando função para renomear valores**

In [85]:
def alteracoes_base(df):
    df['LotShape'] = df['LotShape'].replace({'IR1': 'Ireg', 'IR2': 'Ireg', 'IR3': 'Ireg'})
    
    df['MSZoning'] = df['MSZoning'].replace({'RL':'RL_or_FV', 'FV':'RL_or_FV', 'RM':'RM_or_RH', 'RH':'RM_or_RH'})
    
    df['HouseStyle'] = df['HouseStyle'].replace({'1Story':'1Story_or_1.5', '2Story':'2Story_or_2.5', '1.5Fin':'1Story_or_1.5',
                                                 '1.5Unf':'1Story_or_1.5', '2.5Unf':'2Story_or_2.5', '2.5Fin':'2Story_or_2.5'})

    df['MasVnrType'] = df['MasVnrType'].replace({'BrkFace':'Brk_category', 'BrkCmn':'Brk_category', 0:'None'})
    
    df['BsmtFinType1'] = df['BsmtFinType1'].replace({'BLQ':'BLQ_or_Rec', 'Rec':'BLQ_or_Rec'})

    df['Neighborhood'] = df['Neighborhood'].replace({'NoRidge':'Mediana_A', 'NridgHt':'Mediana_A', 'StoneBr':'Mediana_A',
                            'Timber':'Mediana_B', 'Somerst':'Mediana_B', 'Veenker':'Mediana_B',
                            'Crawfor':'Mediana_B', 'ClearCr':'Mediana_B', 'CollgCr':'Mediana_B',
                            'Blmngtn':'Mediana_B', 'NWAmes':'Mediana_B', 'Gilbert':'Mediana_B',
                            'SawyerW':'Mediana_B', 'Crawfor':'Mediana_B',
                            'Mitchel':'Mediana_C',
                            'NPkVill':'Mediana_C', 'NAmes':'Mediana_C', 'SWISU':'Mediana_C', 
                            'Blueste':'Mediana_C', 'Sawyer':'Mediana_C', 'BrkSide':'Mediana_C', 
                            'Edwards':'Mediana_C', 'OldTown':'Mediana_C', 'BrDale':'Mediana_C',
                            'IDOTRR':'Mediana_C', 'MeadowV':'Mediana_C'})
    
    df = df.replace({'No Fireplaces':'NA'})
    df = df.replace({'No':'NA'})
    
    return df

In [86]:
xtrain = alteracoes_base(xtrain)
xtrain = xtrain.reset_index(drop=True)
xtrain

,MSZoning,LotFrontage,LotShape,LotConfig,Neighborhood,HouseStyle,OverallQual,YearBuilt,YearRemodAdd,RoofStyle,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,TotalBsmtSF,HeatingQC,1stFlrSF,2ndFlrSF,GrLivArea,BsmtFullBath,FullBath,KitchenQual,TotRmsAbvGrd,Fireplaces,FireplaceQu,GarageFinish,GarageCars,GarageArea,GarageQual,WoodDeckSF,OpenPorchSF
0,RM_or_RH,21.0,Reg,Inside,Mediana_C,SFoyer,4,1972,1972,Gable,CemntBd,CmentBd,None,0.0,TA,Gd,CBlock,Gd,TA,Av,ALQ,495,630,Gd,630,0,630,1,1,TA,3,0,NA,No Garage,0,0,No Garage,88,0
1,RL_or_FV,65.0,Reg,Inside,Mediana_B,SLvl,7,1980,1980,Gable,Plywood,Plywood,Brk_category,225.0,TA,TA,CBlock,Gd,TA,Av,GLQ,1036,1372,TA,1472,0,1472,1,2,TA,6,2,TA,Unf,2,588,TA,233,48
2,RL_or_FV,120.0,Reg,Corner,Mediana_C,1Story_or_1.5,6,1968,1968,Hip,Wd Sdng,Wd Sdng,Brk_category,216.0,TA,TA,CBlock,Fa,Fa,NA,Unf,0,1392,Gd,1392,0,1392,1,1,TA,5,2,TA,RFn,2,576,TA,0,0
3,RM_or_RH,59.0,Reg,Corner,Mediana_C,1Story_or_1.5,6,1910,2003,Hip,VinylSd,VinylSd,None,0.0,TA,Gd,CBlock,TA,Fa,NA,Unf,0,485,Gd,1001,634,1635,0,1,Gd,5,0,NA,Unf,1,255,Fa,394,0
4,RL_or_FV,120.0,Ireg,Corner,Mediana_B,1Story_or_1.5,5,1972,1972,Hip,MetalSd,MetalSd,None,0.0,TA,Gd,CBlock,TA,TA,Mn,BLQ_or_Rec,360,864,Ex,864,0,864,0,1,TA,5,0,NA,RFn,1,288,TA,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1236,RL_or_FV,82.0,Reg,Inside,Mediana_A,2Story_or_2.5,8,1999,1999,Gable,VinylSd,VinylSd,Brk_category,673.0,Gd,TA,PConc,Gd,TA,Mn,GLQ,1163,1252,Ex,1268,1097,2365,1,2,Gd,8,1,Gd,RFn,3,856,TA,0,128
1237,RL_or_FV,60.0,Reg,Inside,Mediana_C,1Story_or_1.5,4,1950,1995,Gable,VinylSd,HdBoard,None,0.0,TA,TA,CBlock,Gd,TA,NA,BLQ_or_Rec,442,1067,TA,1067,0,1067,0,2,Gd,4,0,NA,Unf,2,436,TA,290,0
1238,RM_or_RH,68.0,Reg,Inside,Mediana_C,1Story_or_1.5,6,1978,1978,Gable,VinylSd,VinylSd,None,0.0,TA,TA,Slab,NA,NA,NA,NA,0,0,TA,1318,584,1902,0,2,TA,8,0,NA,Unf,2,539,TA,0,0
1239,RL_or_FV,65.0,Reg,Inside,Mediana_B,1Story_or_1.5,7,2003,2004,Gable,VinylSd,VinylSd,Brk_category,18.0,Gd,TA,PConc,Gd,TA,Gd,Unf,0,1374,Ex,1557,0,1557,0,2,Gd,7,1,TA,Fin,2,420,TA,143,20


### **OrdinalEncoder na base de treino**

In [87]:
OE_BsmtFinType1 = OrdinalEncoder(categories=[['NA','Unf','LwQ','BLQ_or_Rec','ALQ','GLQ']], handle_unknown='use_encoded_value', unknown_value=-1)
OE_ExterCond = OrdinalEncoder(categories=[['Po','Fa','TA','Gd','Ex']], handle_unknown='use_encoded_value', unknown_value=-1)
OE_BsmtQual = OrdinalEncoder(categories=[['NA','Po','Fa','TA','Gd','Ex']], handle_unknown='use_encoded_value', unknown_value=-1)        
OE_BsmtCond = OrdinalEncoder(categories=[['NA','Po','Fa','TA','Gd','Ex']], handle_unknown='use_encoded_value', unknown_value=-1)
OE_BsmtExposure = OrdinalEncoder(categories=[['NA','No','Mn','Av','Ex']], handle_unknown='use_encoded_value', unknown_value=-1) 
OE_KitchenQual = OrdinalEncoder(categories=[['Po','Fa','TA','Gd','Ex']], handle_unknown='use_encoded_value', unknown_value=-1)
OE_FireplaceQu = OrdinalEncoder(categories=[['NA','Po','Fa','TA','Gd','Ex']], handle_unknown='use_encoded_value', unknown_value=-1)
OE_GarageFinish = OrdinalEncoder(categories=[['NA','Unf','RFn','Fin']], handle_unknown='use_encoded_value', unknown_value=-1) 
OE_ExterQual = OrdinalEncoder(categories=[['Po','Fa','TA','Gd','Ex']], handle_unknown='use_encoded_value', unknown_value=-1)
OE_HeatingQC = OrdinalEncoder(categories=[['Po','Fa','TA','Gd','Ex']], handle_unknown='use_encoded_value', unknown_value=-1)
OE_GarageQual = OrdinalEncoder(categories=[['NA','Po','Fa','TA','Gd','Ex']], handle_unknown='use_encoded_value', unknown_value=-1)
    
OE_BsmtFinType1_ = OE_BsmtFinType1.fit(xtrain[['BsmtFinType1']])
xtrain['BsmtFinType1'] = OE_BsmtFinType1_.transform(xtrain[['BsmtFinType1']])
    
OE_ExterCond_ = OE_ExterCond.fit(xtrain[['ExterCond']])
xtrain['ExterCond'] = OE_ExterCond_.transform(xtrain[['ExterCond']])
    
OE_BsmtQual_ = OE_BsmtQual.fit(xtrain[['BsmtQual']])
xtrain['BsmtQual'] = OE_BsmtQual_.transform(xtrain[['BsmtQual']])
    
OE_BsmtCond_ = OE_BsmtCond.fit(xtrain[['BsmtCond']])
xtrain['BsmtCond'] = OE_BsmtCond_.transform(xtrain[['BsmtCond']])
    
OE_BsmtExposure_ = OE_BsmtExposure.fit(xtrain[['BsmtExposure']])
xtrain['BsmtExposure'] = OE_BsmtExposure_.transform(xtrain[['BsmtExposure']])
    
OE_KitchenQual_ = OE_KitchenQual.fit(xtrain[['KitchenQual']])
xtrain['KitchenQual'] = OE_KitchenQual_.transform(xtrain[['KitchenQual']])
    
OE_FireplaceQu_ = OE_FireplaceQu.fit(xtrain[['FireplaceQu']])
xtrain['FireplaceQu'] = OE_FireplaceQu_.transform(xtrain[['FireplaceQu']])
    
OE_GarageFinish_ = OE_GarageFinish.fit(xtrain[['GarageFinish']])
xtrain['GarageFinish'] = OE_GarageFinish_.transform(xtrain[['GarageFinish']])
    
OE_ExterQual_ = OE_ExterQual.fit(xtrain[['ExterQual']])
xtrain['ExterQual'] = OE_ExterQual_.transform(xtrain[['ExterQual']])
    
OE_HeatingQC_ = OE_HeatingQC.fit(xtrain[['HeatingQC']])
xtrain['HeatingQC'] = OE_HeatingQC_.transform(xtrain[['HeatingQC']])
    
OE_GarageQual_ = OE_GarageQual.fit(xtrain[['GarageQual']])
xtrain['GarageQual'] = OE_GarageQual_.transform(xtrain[['GarageQual']])
    

In [88]:
xtrain

,MSZoning,LotFrontage,LotShape,LotConfig,Neighborhood,HouseStyle,OverallQual,YearBuilt,YearRemodAdd,RoofStyle,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,TotalBsmtSF,HeatingQC,1stFlrSF,2ndFlrSF,GrLivArea,BsmtFullBath,FullBath,KitchenQual,TotRmsAbvGrd,Fireplaces,FireplaceQu,GarageFinish,GarageCars,GarageArea,GarageQual,WoodDeckSF,OpenPorchSF
0,RM_or_RH,21.0,Reg,Inside,Mediana_C,SFoyer,4,1972,1972,Gable,CemntBd,CmentBd,None,0.0,2.0,3.0,CBlock,4.0,3.0,3.0,4.0,495,630,3.0,630,0,630,1,1,2.0,3,0,0.0,-1.0,0,0,-1.0,88,0
1,RL_or_FV,65.0,Reg,Inside,Mediana_B,SLvl,7,1980,1980,Gable,Plywood,Plywood,Brk_category,225.0,2.0,2.0,CBlock,4.0,3.0,3.0,5.0,1036,1372,2.0,1472,0,1472,1,2,2.0,6,2,3.0,1.0,2,588,3.0,233,48
2,RL_or_FV,120.0,Reg,Corner,Mediana_C,1Story_or_1.5,6,1968,1968,Hip,Wd Sdng,Wd Sdng,Brk_category,216.0,2.0,2.0,CBlock,2.0,2.0,0.0,1.0,0,1392,3.0,1392,0,1392,1,1,2.0,5,2,3.0,2.0,2,576,3.0,0,0
3,RM_or_RH,59.0,Reg,Corner,Mediana_C,1Story_or_1.5,6,1910,2003,Hip,VinylSd,VinylSd,None,0.0,2.0,3.0,CBlock,3.0,2.0,0.0,1.0,0,485,3.0,1001,634,1635,0,1,3.0,5,0,0.0,1.0,1,255,2.0,394,0
4,RL_or_FV,120.0,Ireg,Corner,Mediana_B,1Story_or_1.5,5,1972,1972,Hip,MetalSd,MetalSd,None,0.0,2.0,3.0,CBlock,3.0,3.0,2.0,3.0,360,864,4.0,864,0,864,0,1,2.0,5,0,0.0,2.0,1,288,3.0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1236,RL_or_FV,82.0,Reg,Inside,Mediana_A,2Story_or_2.5,8,1999,1999,Gable,VinylSd,VinylSd,Brk_category,673.0,3.0,2.0,PConc,4.0,3.0,2.0,5.0,1163,1252,4.0,1268,1097,2365,1,2,3.0,8,1,4.0,2.0,3,856,3.0,0,128
1237,RL_or_FV,60.0,Reg,Inside,Mediana_C,1Story_or_1.5,4,1950,1995,Gable,VinylSd,HdBoard,None,0.0,2.0,2.0,CBlock,4.0,3.0,0.0,3.0,442,1067,2.0,1067,0,1067,0,2,3.0,4,0,0.0,1.0,2,436,3.0,290,0
1238,RM_or_RH,68.0,Reg,Inside,Mediana_C,1Story_or_1.5,6,1978,1978,Gable,VinylSd,VinylSd,None,0.0,2.0,2.0,Slab,0.0,0.0,0.0,0.0,0,0,2.0,1318,584,1902,0,2,2.0,8,0,0.0,1.0,2,539,3.0,0,0
1239,RL_or_FV,65.0,Reg,Inside,Mediana_B,1Story_or_1.5,7,2003,2004,Gable,VinylSd,VinylSd,Brk_category,18.0,3.0,2.0,PConc,4.0,3.0,-1.0,1.0,0,1374,4.0,1557,0,1557,0,2,3.0,7,1,3.0,3.0,2,420,3.0,143,20


### **OneHotEncoder na base de treino**

In [89]:
from sklearn.preprocessing import OneHotEncoder

In [90]:
OHE_1 = OneHotEncoder(categories=[['RL_or_FV','RM_or_RH']], handle_unknown='ignore')
OHE_MSZoning = OHE_1.fit(xtrain[['MSZoning']])
df_OHE_MSZoning = pd.DataFrame(OHE_MSZoning.transform(xtrain[['MSZoning']]).toarray(), columns = OHE_MSZoning.get_feature_names_out(),dtype='int64')
xtrain = pd.concat([xtrain,df_OHE_MSZoning],axis=1)
xtrain = xtrain.drop('MSZoning',axis=1)
df_OHE_MSZoning.head(3)

,MSZoning_RL_or_FV,MSZoning_RM_or_RH
0,0,1
1,1,0
2,1,0


In [91]:
OHE_2 = OneHotEncoder(categories=[['Reg','Ireg']], handle_unknown='ignore')
OHE_LotShape = OHE_2.fit(xtrain[['LotShape']])
df_OHE_LotShape = pd.DataFrame(OHE_LotShape.transform(xtrain[['LotShape']]).toarray(), columns = OHE_LotShape.get_feature_names_out(),dtype='int64')
xtrain = pd.concat([xtrain,df_OHE_LotShape],axis=1)
xtrain = xtrain.drop('LotShape',axis=1)
df_OHE_LotShape.head(3)

,LotShape_Reg,LotShape_Ireg
0,1,0
1,1,0
2,1,0


In [92]:
OHE_3 = OneHotEncoder(categories=[['Inside','Corner','CulDSac']], handle_unknown='ignore')
OHE_LotConfig = OHE_3.fit(xtrain[['LotConfig']])
df_OHE_LotConfig = pd.DataFrame(OHE_LotConfig.transform(xtrain[['LotConfig']]).toarray(), columns = OHE_LotConfig.get_feature_names_out(),dtype='int64')
xtrain = pd.concat([xtrain,df_OHE_LotConfig],axis=1)
xtrain = xtrain.drop('LotConfig',axis=1)
df_OHE_LotConfig.head(3)

,LotConfig_Inside,LotConfig_Corner,LotConfig_CulDSac
0,1,0,0
1,1,0,0
2,0,1,0


In [93]:
OHE_4 = OneHotEncoder(categories=[['1Story_or_1.5','2Story_or_2.5']], handle_unknown= 'infrequent_if_exist')
OHE_HouseStyle = OHE_4.fit(xtrain[['HouseStyle']])
df_OHE_HouseStyle = pd.DataFrame(OHE_HouseStyle.transform(xtrain[['HouseStyle']]).toarray(), columns = OHE_HouseStyle.get_feature_names_out(),dtype='int64')
xtrain = pd.concat([xtrain,df_OHE_HouseStyle],axis=1)
xtrain = xtrain.drop('HouseStyle',axis=1)
df_OHE_HouseStyle.head(3)

,HouseStyle_1Story_or_1.5,HouseStyle_2Story_or_2.5
0,0,0
1,0,0
2,1,0


In [94]:
OHE_5 = OneHotEncoder(categories=[['Mediana_A','Mediana_B','Mediana_C']], handle_unknown='ignore')
OHE_Neighborhood = OHE_5.fit(xtrain[['Neighborhood']])
df_OHE_Neighborhood = pd.DataFrame(OHE_Neighborhood.transform(xtrain[['Neighborhood']]).toarray(), columns = OHE_Neighborhood.get_feature_names_out(),dtype='int64')
xtrain = pd.concat([xtrain,df_OHE_Neighborhood],axis=1)
xtrain = xtrain.drop('Neighborhood',axis=1)
df_OHE_Neighborhood.head(3)

,Neighborhood_Mediana_A,Neighborhood_Mediana_B,Neighborhood_Mediana_C
0,0,0,1
1,0,1,0
2,0,0,1


In [95]:
OHE_6 = OneHotEncoder(categories=[['Gable','Hip']], handle_unknown='ignore')
OHE_RoofStyle = OHE_6.fit(xtrain[['RoofStyle']])
df_OHE_RoofStyle = pd.DataFrame(OHE_RoofStyle.transform(xtrain[['RoofStyle']]).toarray(), columns = OHE_RoofStyle.get_feature_names_out(),dtype='int64')
xtrain = pd.concat([xtrain,df_OHE_RoofStyle],axis=1)
xtrain = xtrain.drop('RoofStyle',axis=1)
df_OHE_RoofStyle.head(3)

,RoofStyle_Gable,RoofStyle_Hip
0,1,0
1,1,0
2,0,1


In [96]:
OHE_7 = OneHotEncoder(categories=[['VinylSd','HdBoard','MetalSd','Wd Sdng','Plywood']], handle_unknown='ignore')
OHE_Exterior1st = OHE_7.fit(xtrain[['Exterior1st']])
df_OHE_Exterior1st = pd.DataFrame(OHE_Exterior1st.transform(xtrain[['Exterior1st']]).toarray(), columns = OHE_Exterior1st.get_feature_names_out(),dtype='int64')
xtrain = pd.concat([xtrain,df_OHE_Exterior1st],axis=1)
xtrain = xtrain.drop('Exterior1st',axis=1)
df_OHE_Exterior1st.head(3)

,Exterior1st_VinylSd,Exterior1st_HdBoard,Exterior1st_MetalSd,Exterior1st_Wd Sdng,Exterior1st_Plywood
0,0,0,0,0,0
1,0,0,0,0,1
2,0,0,0,1,0


In [97]:
OHE_8 = OneHotEncoder(categories=[['VinylSd','MetalSd','HdBoard','Wd Sdng','Plywood']], handle_unknown='ignore')
OHE_Exterior2nd = OHE_8.fit(xtrain[['Exterior2nd']])
df_OHE_Exterior2nd = pd.DataFrame(OHE_Exterior2nd.transform(xtrain[['Exterior2nd']]).toarray(), columns = OHE_Exterior2nd.get_feature_names_out(),dtype='int64')
xtrain = pd.concat([xtrain,df_OHE_Exterior2nd],axis=1)
xtrain = xtrain.drop('Exterior2nd',axis=1)
df_OHE_Exterior2nd.head(3)

,Exterior2nd_VinylSd,Exterior2nd_MetalSd,Exterior2nd_HdBoard,Exterior2nd_Wd Sdng,Exterior2nd_Plywood
0,0,0,0,0,0
1,0,0,0,0,1
2,0,0,0,1,0


In [98]:
OHE_9 = OneHotEncoder(categories=[['Stone','Brk_category','None']], handle_unknown='ignore')
OHE_MasVnrType = OHE_9.fit(xtrain[['MasVnrType']])
df_OHE_MasVnrType = pd.DataFrame(OHE_MasVnrType.transform(xtrain[['MasVnrType']]).toarray(), columns = OHE_MasVnrType.get_feature_names_out(),dtype='int64')
xtrain = pd.concat([xtrain,df_OHE_MasVnrType],axis=1)
xtrain = xtrain.drop('MasVnrType',axis=1)
df_OHE_MasVnrType.head(3)

,MasVnrType_Stone,MasVnrType_Brk_category,MasVnrType_None
0,0,0,1
1,0,1,0
2,0,1,0


In [99]:
OHE_10 = OneHotEncoder(categories=[['PConc','CBlock','BrkTil']], handle_unknown='ignore')
OHE_Foundation = OHE_10.fit(xtrain[['Foundation']])
df_OHE_Foundation = pd.DataFrame(OHE_Foundation.transform(xtrain[['Foundation']]).toarray(), columns = OHE_Foundation.get_feature_names_out(),dtype='int64')
xtrain = pd.concat([xtrain,df_OHE_Foundation],axis=1)
xtrain = xtrain.drop('Foundation',axis=1)
df_OHE_Foundation.head(3)

,Foundation_PConc,Foundation_CBlock,Foundation_BrkTil
0,0,1,0
1,0,1,0
2,0,1,0


In [100]:
xtrain

,LotFrontage,OverallQual,YearBuilt,YearRemodAdd,MasVnrArea,ExterQual,ExterCond,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,TotalBsmtSF,HeatingQC,1stFlrSF,2ndFlrSF,GrLivArea,BsmtFullBath,FullBath,KitchenQual,TotRmsAbvGrd,Fireplaces,FireplaceQu,GarageFinish,GarageCars,GarageArea,GarageQual,WoodDeckSF,OpenPorchSF,MSZoning_RL_or_FV,MSZoning_RM_or_RH,LotShape_Reg,LotShape_Ireg,LotConfig_Inside,LotConfig_Corner,LotConfig_CulDSac,HouseStyle_1Story_or_1.5,HouseStyle_2Story_or_2.5,Neighborhood_Mediana_A,Neighborhood_Mediana_B,Neighborhood_Mediana_C,RoofStyle_Gable,RoofStyle_Hip,Exterior1st_VinylSd,Exterior1st_HdBoard,Exterior1st_MetalSd,Exterior1st_Wd Sdng,Exterior1st_Plywood,Exterior2nd_VinylSd,Exterior2nd_MetalSd,Exterior2nd_HdBoard,Exterior2nd_Wd Sdng,Exterior2nd_Plywood,MasVnrType_Stone,MasVnrType_Brk_category,MasVnrType_None,Foundation_PConc,Foundation_CBlock,Foundation_BrkTil
0,21.0,4,1972,1972,0.0,2.0,3.0,4.0,3.0,3.0,4.0,495,630,3.0,630,0,630,1,1,2.0,3,0,0.0,-1.0,0,0,-1.0,88,0,0,1,1,0,1,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0
1,65.0,7,1980,1980,225.0,2.0,2.0,4.0,3.0,3.0,5.0,1036,1372,2.0,1472,0,1472,1,2,2.0,6,2,3.0,1.0,2,588,3.0,233,48,1,0,1,0,1,0,0,0,0,0,1,0,1,0,0,0,0,0,1,0,0,0,0,1,0,1,0,0,1,0
2,120.0,6,1968,1968,216.0,2.0,2.0,2.0,2.0,0.0,1.0,0,1392,3.0,1392,0,1392,1,1,2.0,5,2,3.0,2.0,2,576,3.0,0,0,1,0,1,0,0,1,0,1,0,0,0,1,0,1,0,0,0,1,0,0,0,0,1,0,0,1,0,0,1,0
3,59.0,6,1910,2003,0.0,2.0,3.0,3.0,2.0,0.0,1.0,0,485,3.0,1001,634,1635,0,1,3.0,5,0,0.0,1.0,1,255,2.0,394,0,0,1,1,0,0,1,0,1,0,0,0,1,0,1,1,0,0,0,0,1,0,0,0,0,0,0,1,0,1,0
4,120.0,5,1972,1972,0.0,2.0,3.0,3.0,3.0,2.0,3.0,360,864,4.0,864,0,864,0,1,2.0,5,0,0.0,2.0,1,288,3.0,0,0,1,0,0,1,0,1,0,1,0,0,1,0,0,1,0,0,1,0,0,0,1,0,0,0,0,0,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1236,82.0,8,1999,1999,673.0,3.0,2.0,4.0,3.0,2.0,5.0,1163,1252,4.0,1268,1097,2365,1,2,3.0,8,1,4.0,2.0,3,856,3.0,0,128,1,0,1,0,1,0,0,0,1,1,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,1,0,1,0,0
1237,60.0,4,1950,1995,0.0,2.0,2.0,4.0,3.0,0.0,3.0,442,1067,2.0,1067,0,1067,0,2,3.0,4,0,0.0,1.0,2,436,3.0,290,0,1,0,1,0,1,0,0,1,0,0,0,1,1,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,1,0
1238,68.0,6,1978,1978,0.0,2.0,2.0,0.0,0.0,0.0,0.0,0,0,2.0,1318,584,1902,0,2,2.0,8,0,0.0,1.0,2,539,3.0,0,0,0,1,1,0,1,0,0,1,0,0,0,1,1,0,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0
1239,65.0,7,2003,2004,18.0,3.0,2.0,4.0,3.0,-1.0,1.0,0,1374,4.0,1557,0,1557,0,2,3.0,7,1,3.0,3.0,2,420,3.0,143,20,1,0,1,0,1,0,0,1,0,0,1,0,1,0,1,0,0,0,0,1,0,0,0,0,0,1,0,1,0,0


- robust_cols = ['LotFrontage', 'TotalBsmtSF', '1stFlrSF', 'GrLivArea', 'GarageArea']
- power_cols = ['YearBuilt', 'YearRemodAdd', 'MasVnrArea', 'BsmtFinSF1', 'WoodDeckSF', 'OpenPorchSF', '2ndFlrSF']
- minmax_cols = ['OverallQual', 'Fireplaces', 'GarageCars', 'BsmtFullBath', 'FullBath', 'TotRmsAbvGrd']

# **Validação**

### **Aplicando a função "alterações_base" nos dados de validação**

In [101]:
xtest = alteracoes_base(xtest)
xtest = xtest.reset_index(drop=True)
xtest

,MSZoning,LotFrontage,LotShape,LotConfig,Neighborhood,HouseStyle,OverallQual,YearBuilt,YearRemodAdd,RoofStyle,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,TotalBsmtSF,HeatingQC,1stFlrSF,2ndFlrSF,GrLivArea,BsmtFullBath,FullBath,KitchenQual,TotRmsAbvGrd,Fireplaces,FireplaceQu,GarageFinish,GarageCars,GarageArea,GarageQual,WoodDeckSF,OpenPorchSF
0,RL_or_FV,50.0,Ireg,CulDSac,Mediana_B,1Story_or_1.5,6,1957,1975,Hip,Wd Sdng,Stone,0,0.0,Gd,TA,PConc,TA,TA,NA,BLQ_or_Rec,1219,2035,TA,2515,0,2515,1,3,TA,9,2,TA,RFn,2,484,TA,0,0
1,RL_or_FV,79.0,Reg,Inside,Mediana_C,1Story_or_1.5,6,1941,1950,Gable,Wd Sdng,Wd Sdng,None,0.0,TA,TA,CBlock,TA,TA,NA,BLQ_or_Rec,403,806,TA,958,620,1578,1,1,Fa,5,2,TA,Unf,1,240,TA,0,0
2,RL_or_FV,96.0,Ireg,Corner,Mediana_C,1Story_or_1.5,5,1950,1950,Gable,MetalSd,MetalSd,Brk_category,161.0,TA,TA,CBlock,TA,TA,NA,LwQ,185,709,TA,979,224,1203,1,1,Gd,5,1,TA,Unf,1,352,TA,0,0
3,RL_or_FV,83.0,Reg,Inside,Mediana_B,2Story_or_2.5,7,1977,1977,Hip,Plywood,Plywood,Brk_category,299.0,TA,TA,CBlock,Gd,TA,NA,BLQ_or_Rec,392,1160,Ex,1156,866,2022,0,2,TA,8,1,TA,Fin,2,505,TA,288,117
4,RM_or_RH,21.0,Reg,Inside,Mediana_C,2Story_or_2.5,6,1971,1971,Gable,HdBoard,ImStucc,Brk_category,381.0,TA,TA,CBlock,TA,TA,NA,Unf,0,525,TA,525,567,1092,0,1,TA,6,0,NA,Unf,1,264,TA,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
214,RL_or_FV,68.0,Ireg,Inside,Mediana_B,1Story_or_1.5,9,1981,1987,Gable,WdShing,Wd Shng,None,0.0,Gd,TA,CBlock,Ex,TA,Gd,GLQ,1810,1842,Gd,1842,0,1842,2,0,Gd,5,1,Gd,Fin,3,894,TA,857,72
215,RL_or_FV,71.0,Ireg,Inside,Mediana_C,2Story_or_2.5,6,1965,1979,Flat,Plywood,Plywood,Brk_category,113.0,TA,Gd,CBlock,Gd,TA,NA,LwQ,187,1021,TA,1465,915,2380,0,2,TA,7,1,Po,Unf,2,596,TA,0,265
216,RL_or_FV,85.0,Reg,Inside,Mediana_B,1Story_or_1.5,7,1977,1977,Hip,Plywood,Plywood,Brk_category,209.0,TA,Gd,CBlock,TA,TA,NA,ALQ,822,1386,TA,1411,0,1411,0,2,TA,6,1,TA,Fin,2,544,TA,192,0
217,RL_or_FV,86.0,Ireg,Inside,Mediana_B,2Story_or_2.5,7,1986,1987,Gable,Plywood,Plywood,Brk_category,172.0,Gd,TA,CBlock,TA,TA,Gd,LwQ,28,1502,Ex,1553,1177,2730,1,2,Gd,8,1,TA,Fin,2,576,TA,201,96


### **OrdinalEncoder - Transformando os valores de validação**

In [102]:
xtest['BsmtFinType1'] = OE_BsmtFinType1_.transform(xtest[['BsmtFinType1']])
xtest['ExterCond'] = OE_ExterCond_.transform(xtest[['ExterCond']])
xtest['BsmtQual'] = OE_BsmtQual_.transform(xtest[['BsmtQual']])
xtest['BsmtCond'] = OE_BsmtCond_.transform(xtest[['BsmtCond']])
xtest['BsmtExposure'] = OE_BsmtExposure_.transform(xtest[['BsmtExposure']])
xtest['KitchenQual'] = OE_KitchenQual_.transform(xtest[['KitchenQual']])
xtest['FireplaceQu'] = OE_FireplaceQu_.transform(xtest[['FireplaceQu']])
xtest['GarageFinish'] = OE_GarageFinish_.transform(xtest[['GarageFinish']])
xtest['ExterQual'] = OE_ExterQual_.transform(xtest[['ExterQual']])
xtest['HeatingQC'] = OE_HeatingQC_.transform(xtest[['HeatingQC']])
xtest['GarageQual'] = OE_GarageQual_.transform(xtest[['GarageQual']])

### **OneHotEncoder - Transformando os valores de validação**

In [103]:
df_OHE_MSZoning = pd.DataFrame(OHE_MSZoning.transform(xtest[['MSZoning']]).toarray(), columns = OHE_MSZoning.get_feature_names_out(),dtype='int64')
xtest = pd.concat([xtest,df_OHE_MSZoning],axis=1)
xtest = xtest.drop('MSZoning',axis=1)

df_OHE_LotShape = pd.DataFrame(OHE_LotShape.transform(xtest[['LotShape']]).toarray(), columns = OHE_LotShape.get_feature_names_out(),dtype='int64')
xtest = pd.concat([xtest,df_OHE_LotShape],axis=1)
xtest = xtest.drop('LotShape',axis=1)

df_OHE_LotConfig = pd.DataFrame(OHE_LotConfig.transform(xtest[['LotConfig']]).toarray(), columns = OHE_LotConfig.get_feature_names_out(),dtype='int64')
xtest = pd.concat([xtest,df_OHE_LotConfig],axis=1)
xtest = xtest.drop('LotConfig',axis=1)

df_OHE_HouseStyle = pd.DataFrame(OHE_HouseStyle.transform(xtest[['HouseStyle']]).toarray(), columns = OHE_HouseStyle.get_feature_names_out(),dtype='int64')
xtest = pd.concat([xtest,df_OHE_HouseStyle],axis=1)
xtest = xtest.drop('HouseStyle',axis=1)

df_OHE_Neighborhood = pd.DataFrame(OHE_Neighborhood.transform(xtest[['Neighborhood']]).toarray(), columns = OHE_Neighborhood.get_feature_names_out(),dtype='int64')
xtest = pd.concat([xtest,df_OHE_Neighborhood],axis=1)
xtest = xtest.drop('Neighborhood',axis=1)

df_OHE_RoofStyle = pd.DataFrame(OHE_RoofStyle.transform(xtest[['RoofStyle']]).toarray(), columns = OHE_RoofStyle.get_feature_names_out(),dtype='int64')
xtest = pd.concat([xtest,df_OHE_RoofStyle],axis=1)
xtest = xtest.drop('RoofStyle',axis=1)

df_OHE_Exterior1st = pd.DataFrame(OHE_Exterior1st.transform(xtest[['Exterior1st']]).toarray(), columns = OHE_Exterior1st.get_feature_names_out(),dtype='int64')
xtest = pd.concat([xtest,df_OHE_Exterior1st],axis=1)
xtest = xtest.drop('Exterior1st',axis=1)

df_OHE_Exterior2nd = pd.DataFrame(OHE_Exterior2nd.transform(xtest[['Exterior2nd']]).toarray(), columns = OHE_Exterior2nd.get_feature_names_out(),dtype='int64')
xtest = pd.concat([xtest,df_OHE_Exterior2nd],axis=1)
xtest = xtest.drop('Exterior2nd',axis=1)

df_OHE_MasVnrType = pd.DataFrame(OHE_MasVnrType.transform(xtest[['MasVnrType']]).toarray(), columns = OHE_MasVnrType.get_feature_names_out(),dtype='int64')
xtest = pd.concat([xtest,df_OHE_MasVnrType],axis=1)
xtest = xtest.drop('MasVnrType',axis=1)

df_OHE_Foundation = pd.DataFrame(OHE_Foundation.transform(xtest[['Foundation']]).toarray(), columns = OHE_Foundation.get_feature_names_out(),dtype='int64')
xtest = pd.concat([xtest,df_OHE_Foundation],axis=1)
xtest = xtest.drop('Foundation',axis=1)

In [104]:
# Visualizando a base com as mudanças
xtest

,LotFrontage,OverallQual,YearBuilt,YearRemodAdd,MasVnrArea,ExterQual,ExterCond,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,TotalBsmtSF,HeatingQC,1stFlrSF,2ndFlrSF,GrLivArea,BsmtFullBath,FullBath,KitchenQual,TotRmsAbvGrd,Fireplaces,FireplaceQu,GarageFinish,GarageCars,GarageArea,GarageQual,WoodDeckSF,OpenPorchSF,MSZoning_RL_or_FV,MSZoning_RM_or_RH,LotShape_Reg,LotShape_Ireg,LotConfig_Inside,LotConfig_Corner,LotConfig_CulDSac,HouseStyle_1Story_or_1.5,HouseStyle_2Story_or_2.5,Neighborhood_Mediana_A,Neighborhood_Mediana_B,Neighborhood_Mediana_C,RoofStyle_Gable,RoofStyle_Hip,Exterior1st_VinylSd,Exterior1st_HdBoard,Exterior1st_MetalSd,Exterior1st_Wd Sdng,Exterior1st_Plywood,Exterior2nd_VinylSd,Exterior2nd_MetalSd,Exterior2nd_HdBoard,Exterior2nd_Wd Sdng,Exterior2nd_Plywood,MasVnrType_Stone,MasVnrType_Brk_category,MasVnrType_None,Foundation_PConc,Foundation_CBlock,Foundation_BrkTil
0,50.0,6,1957,1975,0.0,3.0,2.0,3.0,3.0,0.0,3.0,1219,2035,2.0,2515,0,2515,1,3,2.0,9,2,3.0,2.0,2,484,3.0,0,0,1,0,0,1,0,0,1,1,0,0,1,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0
1,79.0,6,1941,1950,0.0,2.0,2.0,3.0,3.0,0.0,3.0,403,806,2.0,958,620,1578,1,1,1.0,5,2,3.0,1.0,1,240,3.0,0,0,1,0,1,0,1,0,0,1,0,0,0,1,1,0,0,0,0,1,0,0,0,0,1,0,0,0,1,0,1,0
2,96.0,5,1950,1950,161.0,2.0,2.0,3.0,3.0,0.0,2.0,185,709,2.0,979,224,1203,1,1,3.0,5,1,3.0,1.0,1,352,3.0,0,0,1,0,0,1,0,1,0,1,0,0,0,1,1,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,1,0
3,83.0,7,1977,1977,299.0,2.0,2.0,4.0,3.0,0.0,3.0,392,1160,4.0,1156,866,2022,0,2,2.0,8,1,3.0,3.0,2,505,3.0,288,117,1,0,1,0,1,0,0,0,1,0,1,0,0,1,0,0,0,0,1,0,0,0,0,1,0,1,0,0,1,0
4,21.0,6,1971,1971,381.0,2.0,2.0,3.0,3.0,0.0,1.0,0,525,2.0,525,567,1092,0,1,2.0,6,0,0.0,1.0,1,264,3.0,0,0,0,1,1,0,1,0,0,0,1,0,0,1,1,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
214,68.0,9,1981,1987,0.0,3.0,2.0,5.0,3.0,-1.0,5.0,1810,1842,3.0,1842,0,1842,2,0,3.0,5,1,4.0,3.0,3,894,3.0,857,72,1,0,0,1,1,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0
215,71.0,6,1965,1979,113.0,2.0,3.0,4.0,3.0,0.0,2.0,187,1021,2.0,1465,915,2380,0,2,2.0,7,1,1.0,1.0,2,596,3.0,0,265,1,0,0,1,1,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,1,0,0,1,0
216,85.0,7,1977,1977,209.0,2.0,3.0,3.0,3.0,0.0,4.0,822,1386,2.0,1411,0,1411,0,2,2.0,6,1,3.0,3.0,2,544,3.0,192,0,1,0,1,0,1,0,0,1,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,1,0,1,0,0,1,0
217,86.0,7,1986,1987,172.0,3.0,2.0,3.0,3.0,-1.0,2.0,28,1502,4.0,1553,1177,2730,1,2,3.0,8,1,3.0,3.0,2,576,3.0,201,96,1,0,0,1,1,0,0,0,1,0,1,0,1,0,0,0,0,0,1,0,0,0,0,1,0,1,0,0,1,0


### **Criando o modelo de Árvore de Regressão**

In [105]:
from sklearn.ensemble import RandomForestRegressor              # Mmodelo: Árvore de Decisão de Regressão
from sklearn.metrics import mean_absolute_error                 # Métrica: Erro médio absoluto
from sklearn.feature_selection import SelectKBest, f_regression # Seletor de features

In [106]:
ytest = ytest.reset_index(drop=True)
ytest

0      200624
1      133000
2      110000
3      192000
4       88000
        ...  
214    385000
215    129000
216     82500
217    301000
218    249700
Name: SalePrice, Length: 219, dtype: int64

In [107]:
# Primeiro modelo, sem hiperparâmetros
rf = RandomForestRegressor(random_state=0)
rf = rf.fit(xtrain, ytrain)
yprev = rf.predict(xtest)
mean_absolute_error(ytest, yprev)

16449.687625570776

### **Pegando as features mais influentes**

In [108]:
feature_names = xtrain.columns
k_best_selector = SelectKBest(score_func=f_regression, k=25)
X_train_k_best = k_best_selector.fit_transform(xtrain, ytrain)
selected_feature_indices = k_best_selector.get_support(indices=True)
selected_feature_names = [feature_names[i] for i in selected_feature_indices]
print(selected_feature_names)

xtrain_best_features = xtrain[selected_feature_names].copy()
xtest_best_features = xtest[selected_feature_names].copy()

['OverallQual', 'YearBuilt', 'YearRemodAdd', 'MasVnrArea', 'ExterQual', 'BsmtQual', 'BsmtFinSF1', 'TotalBsmtSF', 'HeatingQC', '1stFlrSF', 'GrLivArea', 'FullBath', 'KitchenQual', 'TotRmsAbvGrd', 'Fireplaces', 'FireplaceQu', 'GarageFinish', 'GarageCars', 'GarageArea', 'Neighborhood_Mediana_A', 'Neighborhood_Mediana_C', 'MasVnrType_Stone', 'MasVnrType_None', 'Foundation_PConc', 'Foundation_CBlock']


### **Criando o modelo alterando alguns hiperparâmetros**
- Obs: Nessa parte temos a opção de usar o GridSearch para retornar os melhores hiperparâmetros baseado em uma métrica passada. Ele faz diversas combinações com os dados que passamos através de um dicionário com o nome e diferentes valores dos hiperparâmetros.

In [109]:
rf = RandomForestRegressor(n_estimators=140,max_depth=11, random_state=10, max_features=6,criterion='absolute_error')

#### **Validação cruzada nos dados de treino**

In [110]:
from sklearn.model_selection import cross_val_score, KFold
kf = KFold(n_splits=5, shuffle=True) # Shuffle: embaralha
rf_scores = cross_val_score(rf, xtrain, ytrain, cv=kf)
print("Média dos Scores:", np.mean(rf_scores))

Média dos Scores: 0.8688633551355412


#### **Erro médio absoluto do ytest e a previsão do modelo**

In [111]:
rf = RandomForestRegressor(n_estimators=140,max_depth=11, random_state=10, max_features=6,criterion='absolute_error')
rf = rf.fit(xtrain_best_features, ytrain)
yprev = rf.predict(xtest_best_features)
mean_absolute_error(ytest, yprev)

15939.233300717548

#### **Visualizando o SalePrice dos registros de validação e a previsão do modelo**

In [112]:
comparacao_saleprice = pd.DataFrame({'SalePrice':ytest, 'Previsão':yprev})
comparacao_saleprice.sample(15, random_state=0)

,SalePrice,Previsão
152,239000,290438.732143
74,112000,106401.250000
71,118000,132274.750000
161,142000,144276.250000
162,197000,189533.610714
143,150500,149972.553571
63,180500,150485.528571
153,145900,149636.560714
218,249700,246496.371429
135,213250,240842.889286


### **Colocando em prática na base de teste do case**

In [113]:
# Importando a base de teste
base_teste = pd.read_csv("../dados/test.csv")
base_teste

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,1461,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,NAmes,Feedr,Norm,1Fam,1Story,5,6,1961,1961,Gable,CompShg,VinylSd,VinylSd,None,0.0,TA,TA,CBlock,TA,TA,No,Rec,468.0,LwQ,144.0,270.0,882.0,GasA,TA,Y,SBrkr,896,0,0,896,0.0,0.0,1,0,2,1,TA,5,Typ,0,NaN,Attchd,1961.0,Unf,1.0,730.0,TA,TA,Y,140,0,0,0,120,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal
1,1462,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,NAmes,Norm,Norm,1Fam,1Story,6,6,1958,1958,Hip,CompShg,Wd Sdng,Wd Sdng,BrkFace,108.0,TA,TA,CBlock,TA,TA,No,ALQ,923.0,Unf,0.0,406.0,1329.0,GasA,TA,Y,SBrkr,1329,0,0,1329,0.0,0.0,1,1,3,1,Gd,6,Typ,0,NaN,Attchd,1958.0,Unf,1.0,312.0,TA,TA,Y,393,36,0,0,0,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal
2,1463,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,Gilbert,Norm,Norm,1Fam,2Story,5,5,1997,1998,Gable,CompShg,VinylSd,VinylSd,None,0.0,TA,TA,PConc,Gd,TA,No,GLQ,791.0,Unf,0.0,137.0,928.0,GasA,Gd,Y,SBrkr,928,701,0,1629,0.0,0.0,2,1,3,1,TA,6,Typ,1,TA,Attchd,1997.0,Fin,2.0,482.0,TA,TA,Y,212,34,0,0,0,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal
3,1464,60,RL,78.0,9978,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,Gilbert,Norm,Norm,1Fam,2Story,6,6,1998,1998,Gable,CompShg,VinylSd,VinylSd,BrkFace,20.0,TA,TA,PConc,TA,TA,No,GLQ,602.0,Unf,0.0,324.0,926.0,GasA,Ex,Y,SBrkr,926,678,0,1604,0.0,0.0,2,1,3,1,Gd,7,Typ,1,Gd,Attchd,1998.0,Fin,2.0,470.0,TA,TA,Y,360,36,0,0,0,0,NaN,NaN,NaN,0,6,2010,WD,Normal
4,1465,120,RL,43.0,5005,Pave,NaN,IR1,HLS,AllPub,Inside,Gtl,StoneBr,Norm,Norm,TwnhsE,1Story,8,5,1992,1992,Gable,CompShg,HdBoard,HdBoard,None,0.0,Gd,TA,PConc,Gd,TA,No,ALQ,263.0,Unf,0.0,1017.0,1280.0,GasA,Ex,Y,SBrkr,1280,0,0,1280,0.0,0.0,2,0,2,1,Gd,5,Typ,0,NaN,Attchd,1992.0,RFn,2.0,506.0,TA,TA,Y,0,82,0,0,144,0,NaN,NaN,NaN,0,1,2010,WD,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1454,2915,160,RM,21.0,1936,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,MeadowV,Norm,Norm,Twnhs,2Story,4,7,1970,1970,Gable,CompShg,CemntBd,CmentBd,None,0.0,TA,TA,CBlock,TA,TA,No,Unf,0.0,Unf,0.0,546.0,546.0,GasA,Gd,Y,SBrkr,546,546,0,1092,0.0,0.0,1,1,3,1,TA,5,Typ,0,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,Y,0,0,0,0,0,0,NaN,NaN,NaN,0,6,2006,WD,Normal
1455,2916,160,RM,21.0,1894,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,MeadowV,Norm,Norm,TwnhsE,2Story,4,5,1970,1970,Gable,CompShg,CemntBd,CmentBd,None,0.0,TA,TA,CBlock,TA,TA,No,Rec,252.0,Unf,0.0,294.0,546.0,GasA,TA,Y,SBrkr,546,546,0,1092,0.0,0.0,1,1,3,1,TA,6,Typ,0,NaN,CarPort,1970.0,Unf,1.0,286.0,TA,TA,Y,0,24,0,0,0,0,NaN,NaN,NaN,0,4,2006,WD,Abnorml
1456,2917,20,RL,160.0,20000,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,Mitchel,Norm,Norm,1Fam,1Story,5,7,1960,1996,Gable,CompShg,VinylSd,VinylSd,None,0.0,TA,TA,CBlock,TA,TA,No,ALQ,1224.0,Unf,0.0,0.0,1224.0,GasA,Ex,Y,SBrkr,1224,0,0,1224,1.0,0.0,1,0,4,1,TA,7,Typ,1,TA,Detchd,1960.0,Unf,2.0,576.0,TA,TA,Y,474,0,0,0,0,0,NaN,NaN,NaN,0,9,2006,WD,Abnorml
1457,2918,85,RL,62.0,10441,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,Mitchel,Norm,Norm,1Fam,SFoyer,5,5,1992,1992,Gable,Comp

#### **Apagando a coluna "Id"**

In [114]:
base_teste = base_teste.drop('Id',axis=1)
base_teste.head(3)

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,NAmes,Feedr,Norm,1Fam,1Story,5,6,1961,1961,Gable,CompShg,VinylSd,VinylSd,None,0.0,TA,TA,CBlock,TA,TA,No,Rec,468.0,LwQ,144.0,270.0,882.0,GasA,TA,Y,SBrkr,896,0,0,896,0.0,0.0,1,0,2,1,TA,5,Typ,0,NaN,Attchd,1961.0,Unf,1.0,730.0,TA,TA,Y,140,0,0,0,120,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal
1,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,NAmes,Norm,Norm,1Fam,1Story,6,6,1958,1958,Hip,CompShg,Wd Sdng,Wd Sdng,BrkFace,108.0,TA,TA,CBlock,TA,TA,No,ALQ,923.0,Unf,0.0,406.0,1329.0,GasA,TA,Y,SBrkr,1329,0,0,1329,0.0,0.0,1,1,3,1,Gd,6,Typ,0,NaN,Attchd,1958.0,Unf,1.0,312.0,TA,TA,Y,393,36,0,0,0,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal
2,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,Gilbert,Norm,Norm,1Fam,2Story,5,5,1997,1998,Gable,CompShg,VinylSd,VinylSd,None,0.0,TA,TA,PConc,Gd,TA,No,GLQ,791.0,Unf,0.0,137.0,928.0,GasA,Gd,Y,SBrkr,928,701,0,1629,0.0,0.0,2,1,3,1,TA,6,Typ,1,TA,Attchd,1997.0,Fin,2.0,482.0,TA,TA,Y,212,34,0,0,0,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal


### **Criando uma função para aplicar todos pré-processamentos que fiz nos dados de treino e validação**
- Obs: Aqui eu poderia ter feito tudo manualmente, tratando os valores com mais detalhes. Porém, optei por criar essa função para automatizar essas etapas.

In [115]:
def transformar(base_teste):
    
    # Colunas categóricas
    base_teste = base_teste.drop(['Street', 'Utilities', 'LandContour', 'LandSlope', 'Condition1', 'Condition2', 
                'BldgType', 'RoofMatl', 'Heating', 'Electrical', 'Functional', 'GarageType', 
                'GarageCond', 'PavedDrive','CentralAir','SaleType','SaleCondition'],axis=1)
    # Colunas numéricas
    base_teste = base_teste.drop(['KitchenAbvGr', 'EnclosedPorch', 'MSSubClass', 'OverallCond', 'YrSold', 'LowQualFinSF', 
                  'MiscVal', 'BsmtHalfBath', 'BsmtFinSF2', '3SsnPorch', 'MoSold', 'ScreenPorch', 'BedroomAbvGr',
                  'BsmtUnfSF', 'GarageYrBlt', 'LotArea', 'HalfBath','BsmtFinType2'],axis=1)
    
    base_teste = alteracoes_base(base_teste)
    base_teste = base_teste.fillna(0)
    
    base_teste['BsmtFinType1'] = OE_BsmtFinType1_.transform(base_teste[['BsmtFinType1']])
    base_teste['ExterCond'] = OE_ExterCond_.transform(base_teste[['ExterCond']])
    base_teste['BsmtQual'] = OE_BsmtQual_.transform(base_teste[['BsmtQual']])
    base_teste['BsmtCond'] = OE_BsmtCond_.transform(base_teste[['BsmtCond']])
    base_teste['BsmtExposure'] = OE_BsmtExposure_.transform(base_teste[['BsmtExposure']])
    base_teste['KitchenQual'] = OE_KitchenQual_.transform(base_teste[['KitchenQual']])
    base_teste['FireplaceQu'] = OE_FireplaceQu_.transform(base_teste[['FireplaceQu']])
    base_teste['GarageFinish'] = OE_GarageFinish_.transform(base_teste[['GarageFinish']])
    base_teste['ExterQual'] = OE_ExterQual_.transform(base_teste[['ExterQual']])
    base_teste['HeatingQC'] = OE_HeatingQC_.transform(base_teste[['HeatingQC']])
    base_teste['GarageQual'] = OE_GarageQual_.transform(base_teste[['GarageQual']])
    
    df_OHE_MSZoning = pd.DataFrame(OHE_MSZoning.transform(base_teste[['MSZoning']]).toarray(), columns = OHE_MSZoning.get_feature_names_out(),dtype='int64')
    base_teste = pd.concat([base_teste,df_OHE_MSZoning],axis=1)
    base_teste = base_teste.drop('MSZoning',axis=1)

    df_OHE_LotShape = pd.DataFrame(OHE_LotShape.transform(base_teste[['LotShape']]).toarray(), columns = OHE_LotShape.get_feature_names_out(),dtype='int64')
    base_teste = pd.concat([base_teste,df_OHE_LotShape],axis=1)
    base_teste = base_teste.drop('LotShape',axis=1)

    df_OHE_LotConfig = pd.DataFrame(OHE_LotConfig.transform(base_teste[['LotConfig']]).toarray(), columns = OHE_LotConfig.get_feature_names_out(),dtype='int64')
    base_teste = pd.concat([base_teste,df_OHE_LotConfig],axis=1)
    base_teste = base_teste.drop('LotConfig',axis=1)

    df_OHE_HouseStyle = pd.DataFrame(OHE_HouseStyle.transform(base_teste[['HouseStyle']]).toarray(), columns = OHE_HouseStyle.get_feature_names_out(),dtype='int64')
    base_teste = pd.concat([base_teste,df_OHE_HouseStyle],axis=1)
    base_teste = base_teste.drop('HouseStyle',axis=1)

    df_OHE_Neighborhood = pd.DataFrame(OHE_Neighborhood.transform(base_teste[['Neighborhood']]).toarray(), columns = OHE_Neighborhood.get_feature_names_out(),dtype='int64')
    base_teste = pd.concat([base_teste,df_OHE_Neighborhood],axis=1)
    base_teste = base_teste.drop('Neighborhood',axis=1)

    df_OHE_RoofStyle = pd.DataFrame(OHE_RoofStyle.transform(base_teste[['RoofStyle']]).toarray(), columns = OHE_RoofStyle.get_feature_names_out(),dtype='int64')
    base_teste = pd.concat([base_teste,df_OHE_RoofStyle],axis=1)
    base_teste = base_teste.drop('RoofStyle',axis=1)

    df_OHE_Exterior1st = pd.DataFrame(OHE_Exterior1st.transform(base_teste[['Exterior1st']]).toarray(), columns = OHE_Exterior1st.get_feature_names_out(),dtype='int64')
    base_teste = pd.concat([base_teste,df_OHE_Exterior1st],axis=1)
    base_teste = base_teste.drop('Exterior1st',axis=1)

    df_OHE_Exterior2nd = pd.DataFrame(OHE_Exterior2nd.transform(base_teste[['Exterior2nd']]).toarray(), columns = OHE_Exterior2nd.get_feature_names_out(),dtype='int64')
    base_teste = pd.concat([base_teste,df_OHE_Exterior2nd],axis=1)
    base_teste = base_teste.drop('Exterior2nd',axis=1)

    df_OHE_MasVnrType = pd.DataFrame(OHE_MasVnrType.transform(base_teste[['MasVnrType']]).toarray(), columns = OHE_MasVnrType.get_feature_names_out(),dtype='int64')
    base_teste = pd.concat([base_teste,df_OHE_MasVnrType],axis=1)
    base_teste = base_teste.drop('MasVnrType',axis=1)

    df_OHE_Foundation = pd.DataFrame(OHE_Foundation.transform(base_teste[['Foundation']]).toarray(), columns = OHE_Foundation.get_feature_names_out(),dtype='int64')
    base_teste = pd.concat([base_teste,df_OHE_Foundation],axis=1)
    base_teste = base_teste.drop('Foundation',axis=1)
    
    
    base_teste = base_teste[selected_feature_names].copy()
    yprev = rf.predict(base_teste)
    
    base_teste['SalePrice'] = yprev
    
    previsao_final = pd.DataFrame({'Id':np.arange(1461,2920), 'SalePrice':yprev})
    
    return previsao_final

In [116]:
# Tranformando a base de teste
predict = transformar(base_teste)

In [117]:
# Transformando em um arquivo csv para enviar o resultado na competição
# predict.to_csv('predict.csv', index=False)

### **Resultado final**

In [118]:
predict

,Id,SalePrice
0,1461,125126.485714
1,1462,142816.071429
2,1463,179796.996429
3,1464,185842.517857
4,1465,200595.071429
...,...,...
1454,2915,86561.100000
1455,2916,88008.242857
1456,2917,155380.125000
1457,2918,108871.250000
